In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%cd /kaggle/input/amazonreviews

In [ ]:
import bz2 
f = bz2.open("train.ft.txt.bz2", "rt")
train = f.readlines()
 

In [ ]:
train[0]

In [ ]:
import bz2 
f = bz2.open("test.ft.txt.bz2", "rt")
test = f.readlines()  

In [ ]:
import pandas as pd
train_data = pd.DataFrame()

In [ ]:
x_train=[]
y_train=[]
for line in train:
    line=line.split()
    y_train.append(1) if line[0] =="__label__2" else y_train.append(0)
    x_train.append(" ".join(line[1:]))
train_data['reviews'] = x_train
train_data['labels'] = y_train

In [ ]:
test_data['labels']

In [ ]:
import pandas as pd
test_data = pd.DataFrame()

In [ ]:
x_test=[]
y_test=[]
for line in train:
    line=line.split()
    y_test.append(1) if line[0] =="__label__2" else y_test.append(0)
    x_train.append(" ".join(line[1:]))
test_data['reviews'] = x_test
test_data['labels'] = y_test

In [ ]:
X_train = train_data['reviews']
Y_train = train_data['labels']
X_test = test_data['reviews']
Y_test = test_data['labels']

In [ ]:
def preprocess(in_text):
    # If we have html tags, remove them by this way:
    #out_text = remove_tags(in_text)
    # Remove punctuations and numbers
    out_text = re.sub('[^a-zA-Z]', ' ', str(in_text))
    # Convert upper case to lower case
    out_text="".join(list(map(lambda x:x.lower(),out_text)))
    # Remove single character
    out_text = re.sub(r"\s+[a-zA-Z]\s+", ' ', out_text)
    return out_text

In [ ]:
import re
x_train=[]
for review in list(X_train):
    x_train.append(preprocess(review))

In [ ]:
import re
x_test=[]
for review in list(X_test):
    x_test.append(preprocess(review))

In [ ]:
train_data= pd.DataFrame()
train_data['reviews'] = x_train

In [ ]:
Y_train.shape

In [ ]:
train_data['labels'] = list(Y_train)

In [ ]:
test_data= pd.DataFrame()
test_data['reviews'] = x_test
test_data['labels'] = list(Y_test)

In [ ]:
import seaborn as sns
sns.countplot(x='labels', data=train_data)

In [ ]:
x_train = train_data['reviews']
y_train = train_data['labels']
x_test = test_data['reviews']
y_test = test_data['labels']

In [ ]:
x_train=np.array(x_train.values.tolist())
x_test=np.array(x_test.values.tolist())
y_train=np.array(y_train.values.tolist())
y_test=np.array(y_test.values.tolist())

In [ ]:
from keras_preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
word_index=tokenizer.word_index
vocab_size = len(word_index)+1
print(vocab_size)

In [ ]:
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
maxlen = 100
x_train_pad = pad_sequences(x_train, padding='post', maxlen=maxlen)
x_test_pad = pad_sequences(x_test, padding='post', maxlen=maxlen)

In [ ]:
from tensorflow.keras.layers import Flatten, GRU, Dense, Flatten, Embedding
from tensorflow.keras.models import Sequential
model = Sequential()
model.add(Embedding(vocab_size, 20, input_length=maxlen))
model.add(GRU(units=32,dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

In [ ]:
history = model.fit(x_train_pad, y_train, batch_size=128, epochs=10, verbose=1, validation_data=(x_test_pad, y_test))

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()